In [1]:
import warnings
warnings.filterwarnings('ignore')

# Import Necessary libraries
import numpy as np 
import pandas as pd 

# Import Model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

# Import Sampler libraries
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline


In [2]:
# Set the decimal format
pd.options.display.float_format = "{:.2f}".format

In [3]:
# Read the data
df = pd.read_csv("diabetes_prediction_dataset.csv")

In [4]:
# Handle duplicates
df = df.drop_duplicates()

In [5]:
# Remove unnecessary values [0.00195%]
df = df[df['gender'] != 'Other']

In [6]:
# Recategorize smoking history
def recategorize_smoking(smoking_status):
    if smoking_status in ['never', 'No Info']:
        return 'non-smoker'
    elif smoking_status == 'current':
        return 'current'
    elif smoking_status in ['ever', 'former', 'not current']:
        return 'past_smoker'


In [7]:
df['smoking_history'] = df['smoking_history'].apply(recategorize_smoking)

In [8]:
# One-hot encoding
def perform_one_hot_encoding(df, column_name):
    dummies = pd.get_dummies(df[column_name], prefix=column_name)
    df = pd.concat([df.drop(column_name, axis=1), dummies], axis=1)
    return df


In [9]:
data = df.copy()
data = perform_one_hot_encoding(data, 'gender')
data = perform_one_hot_encoding(data, 'smoking_history')

In [10]:
# Resampling
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)

In [11]:
# Feature Selection and PCA
X = data.drop('diabetes', axis=1)
y = data['diabetes']

In [12]:
# You can customize the feature selection and PCA steps based on your specific needs
# Here, I'm using PCA to reduce the dimensionality to 12 features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.select_dtypes(include='number').columns),
        ('cat', OneHotEncoder(), X.select_dtypes(include='object').columns),
        ('pca', PCA(n_components=11), X.columns)  # Adjust the number of components as needed
    ])


In [13]:
# Model Pipeline with Gaussian Naive Bayes (without GridSearchCV)
clf = imbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('over', over),
    ('under', under),
    ('classifier', GaussianNB())
])


In [14]:
param_grid = {
    'classifier__var_smoothing': [0.08112]
}

In [15]:
# Create Grid Search object
grid_search = GridSearchCV(clf, param_grid, cv=5)

In [16]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
# Train the model with Grid Search
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['age', 'hypertension', 'heart_disease', 'bmi', 'HbA1c_level',
       'blood_glucose_level'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         Index([], dtype='object')),
                                                                        ('pca',
                                                                         PCA(n_components=11),
                                                                         Index(['age', 'hypertension', 'heart_disease', 'bmi', 'HbA1c_level',
       'blood_glucose_level', 'gender_Female', 'gender_Male',
       'smoking_history_current', 'smoking_history_non-smoker',
       'smoking_history_past_smoker'],
      dtype='object'))])),
                                       ('over', SMOTE(sampling_strategy=0.1)),
                                       ('under',
                                        RandomUnderSampler(sampling_strategy=0.5)),
                                       ('classifier', GaussianNB())]),
             param_grid={'classifier__var_smoothing': [0.08112]})

In [18]:
# Print the best parameters
print("Best Parameters: ", grid_search.best_params_)

Best Parameters:  {'classifier__var_smoothing': 0.08112}


In [19]:
# Predict on the test set using the best model
y_pred = grid_search.predict(X_test)

In [20]:
# Evaluate the model
print("Model Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Model Accuracy:  0.9135545615312598
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     17525
           1       0.51      0.46      0.48      1701

    accuracy                           0.91     19226
   macro avg       0.73      0.71      0.72     19226
weighted avg       0.91      0.91      0.91     19226

